In [17]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error  # or accuracy_score for classification

# 1) Read in your CSV
df = pd.read_csv("stacking_train.csv")


def parse_label(x):
    # x might be something like "tensor([6.4000], dtype=torch.float64)"
    # or a tuple string "(tensor([6.4000], dtype=...),4)"
    # We'll just grab the last floating‐point number in the string.
    nums = re.findall(r"[\d\.]+", str(x))
    # assume the true label is the last number
    return float(nums[0])


df["image_label"] = df["image_label"].apply(parse_label)

# 2) Pivot so each model’s prediction becomes its own column
wide = df.pivot(
    index="image_name",  # one row per image
    columns="weight_file",  # one column per base model
    values="output",  # the model’s numeric output
).reset_index()

# 3) Bring the ground-truth label back in
#    (assuming you also stored `image_label` in your long DataFrame)
labels = (
    df[["image_name", "image_label"]]
    .drop_duplicates(subset="image_name")
    .set_index("image_name")
)
wide = wide.set_index("image_name").join(labels).reset_index()

# 4) Prepare X, y
X = wide.drop(["image_name", "image_label"], axis=1)
y = wide["image_label"]

# 6) Fit an XGBoost meta-model
#    (choose XGBRegressor if it’s a regression task; XGBClassifier if classification)
meta_model = xgb.XGBRegressor(
    n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42
)
meta_model.fit(X, y)

# # 7) Evaluate
# y_pred = meta_model.predict(X_val)
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# print(f"Validation RMSE: {rmse:.4f}")

# # 8) (Optional) Retrain on the full dataset
# meta_model.fit(X, y)

# # 9) Save your trained meta-model for later
# import joblib
# joblib.dump(meta_model, "xgb_stacker.joblib")

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [20]:
import numpy as np

df_val = pd.read_csv("stacking_val.csv")


def parse_label(x):
    # x might be something like "tensor([6.4000], dtype=torch.float64)"
    # or a tuple string "(tensor([6.4000], dtype=...),4)"
    # We'll just grab the last floating‐point number in the string.
    nums = re.findall(r"[\d\.]+", str(x))
    # assume the true label is the last number
    return float(nums[0])


df_val["image_label"] = df_val["image_label"].apply(parse_label)

# 2) Pivot so each model’s prediction becomes its own column
wide = df_val.pivot(
    index="image_name",  # one row per image
    columns="weight_file",  # one column per base model
    values="output",  # the model’s numeric output
).reset_index()

# 3) Bring the ground-truth label back in
#    (assuming you also stored `image_label` in your long DataFrame)
labels = (
    df_val[["image_name", "image_label"]]
    .drop_duplicates(subset="image_name")
    .set_index("image_name")
)
wide = wide.set_index("image_name").join(labels).reset_index()

# 4) Prepare X, y
X_val = wide.drop(["image_name", "image_label"], axis=1)
y_val = wide["image_label"]

y_pred = meta_model.predict(X_val)

errors = abs(y_val - y_pred)
fraction_within_1 = np.mean(errors <= 1)
print(f"Fraction of predictions with |error| ≤ 1: {fraction_within_1:.3f}")

Fraction of predictions with |error| ≤ 1: 0.838
